In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

fairuzazaria_rtm_stuck_prediction_datasets_idt_path = kagglehub.dataset_download('fairuzazaria/rtm-stuck-prediction-datasets-idt')
fairuzazaria_rtm_stuck_prediction_datasets_idt_updated_path = kagglehub.dataset_download('fairuzazaria/rtm-stuck-prediction-datasets-idt-updated')
fairuzazaria_rtm_stuck_prediction_datasets_idt_adjusted_path = kagglehub.dataset_download('fairuzazaria/rtm-stuck-prediction-datasets-idt-adjusted')

print('Data source import complete.')


# **DATA PREDICTION**

In [ ]:
step_in   = 90
step_out  = 60
scaling   = False
well_name = "generalized"
scale_type = "no_scale"
use_additionals = True

if scaling:
    scale_type = "minmax"

# **1. PREPARATION**

## **1.1 IMPORT LIBRARIES**

In [ ]:
import os

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import gc
import csv
import glob
import h5py
import joblib
import random

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from sklearn import metrics
from sklearn.metrics import auc
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [ ]:
import keras
import tensorflow as tf
import tensorflow.keras.backend as K

from keras.models import Model

from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import Nadam

from keras.optimizers.schedules import ExponentialDecay
from keras.optimizers.schedules import CosineDecay

from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l1_l2

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import BinaryFocalCrossentropy

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.metrics import AUC
from tensorflow.keras.metrics import Recall
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import TruePositives
from tensorflow.keras.metrics import TrueNegatives

from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Attention
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import GlobalAveragePooling1D

2025-06-10 09:57:14.703881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749549434.890922      89 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749549434.950891      89 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from typing import Any
from json import loads, dumps
from datetime import datetime, date

In [ ]:
from tqdm.notebook import tqdm
from multiprocessing import Lock

tqdm.set_lock(Lock())

## **1.2 PREPARE FUNCTIONS**

In [ ]:
def load_h5_data(file_path, filtered=True, with_class = False):
    with h5py.File(file_path, "r") as f:
        X = f["X"][:]
        y = f["y"][:]

    if filtered:
        normal_mask = y == 0
        X = X[normal_mask]

    if with_class:
        return X.astype("float32"), y.astype("float32")

    return X.astype("float32")

In [ ]:
def make_dataset(X, y, batch_size=128, shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
def feature_wise_minmax(X):
    num_samples, timesteps, num_features = X.shape
    X_scaled = np.zeros_like(X)

    scalers = []
    for i in range(num_features):
        scaler  = MinMaxScaler()
        feature = X[:, :, i].reshape(-1, 1)
        X_scaled[:, :, i] = scaler.fit_transform(feature).reshape(num_samples, timesteps)

        scalers.append(scaler)

    return X_scaled, scalers

In [ ]:
def feature_wise_minmax_transform(X, scalers):
    num_samples, timesteps, num_features = X.shape
    X_scaled = np.zeros_like(X)

    for i in range(num_features):
        feature = X[:, :, i].reshape(-1, 1)
        X_scaled[:, :, i] = scalers[i].transform(feature).reshape(num_samples, timesteps)

    return X_scaled

# **2. DATA PREPARATION**

In [ ]:
# -- load datasets
base_path = f"rtm-stuck-prediction-datasets-idt-adjusted/3.idt-adjusted/{step_in}{step_out}"
dataset_train = [
    f"/kaggle/input/{base_path}/well_a_train_adds_normal_{step_in}{step_out}_0_new.h5",
    f"/kaggle/input/{base_path}/well_b_train_adds_normal_{step_in}{step_out}_0_new.h5",
    f"/kaggle/input/{base_path}/well_d_train_adds_normal_{step_in}{step_out}_0_new.h5"
]

dataset_test = [
    f"/kaggle/input/{base_path}/well_a_test_adds_normal_{step_in}{step_out}_0_new.h5",
    f"/kaggle/input/{base_path}/well_b_test_adds_normal_{step_in}{step_out}_0_new.h5",
    f"/kaggle/input/{base_path}/well_d_test_adds_normal_{step_in}{step_out}_0_new.h5"
]

dataset_train, dataset_test

(['/kaggle/input/rtm-stuck-prediction-datasets-idt-adjusted/3.idt-adjusted/9060/well_a_train_adds_normal_9060_0_new.h5',
  '/kaggle/input/rtm-stuck-prediction-datasets-idt-adjusted/3.idt-adjusted/9060/well_b_train_adds_normal_9060_0_new.h5',
  '/kaggle/input/rtm-stuck-prediction-datasets-idt-adjusted/3.idt-adjusted/9060/well_d_train_adds_normal_9060_0_new.h5'],
 ['/kaggle/input/rtm-stuck-prediction-datasets-idt-adjusted/3.idt-adjusted/9060/well_a_test_adds_normal_9060_0_new.h5',
  '/kaggle/input/rtm-stuck-prediction-datasets-idt-adjusted/3.idt-adjusted/9060/well_b_test_adds_normal_9060_0_new.h5',
  '/kaggle/input/rtm-stuck-prediction-datasets-idt-adjusted/3.idt-adjusted/9060/well_d_test_adds_normal_9060_0_new.h5'])

In [ ]:
X_train, y_train, X_test, y_test = [], [], [], []
for i in range(len(dataset_train)):
    X_tr, y_tr = load_h5_data(dataset_train[i], filtered=False, with_class=True)
    X_te, y_te = load_h5_data(dataset_test[i], filtered=False, with_class=True)

    X_train.append(X_tr)
    y_train.append(y_tr)
    X_test.append(X_te)
    y_test.append(y_te)

    print(f'X_train_{i} : {X_tr.shape}')
    print(f'X_test_{i}  : {y_tr.shape}')
    print(f'y_train_{i} : {X_te.shape}')
    print(f'y_test_{i}  : {y_te.shape}\n')

X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)
X_test  = np.concatenate(X_test, axis=0)
y_test  = np.concatenate(y_test, axis=0)

timesteps, n_features = features, steps = X_train.shape[1], X_train.shape[2]

X_train_0 : (622009, 90, 20)
X_test_0  : (622009,)
y_train_0 : (155443, 90, 20)
y_test_0  : (155443,)

X_train_1 : (362357, 90, 20)
X_test_1  : (362357,)
y_train_1 : (96264, 90, 20)
y_test_1  : (96264,)

X_train_2 : (449002, 90, 20)
X_test_2  : (449002,)
y_train_2 : (76087, 90, 20)
y_test_2  : (76087,)



In [ ]:
if scaling:
    X_train_con, X_train_cat = X_train[:, :, :9], X_train[:, :, 9:]
    X_test_con, X_test_cat   = X_test[:, :, :9],  X_test[:, :, 9:]

    print(X_train_con.shape, X_train_cat.shape)
    print(X_test_con.shape, X_test_cat.shape)

In [ ]:
if scaling:
    n_train_samples, n_train_timesteps, _ = X_train_con.shape
    n_test_samples, n_test_timesteps, _   = X_test_con.shape

    X_train_con, scaler = feature_wise_minmax(X_train_con)
    X_test_con = feature_wise_minmax_transform(X_test_con, scaler)

    X_train = np.concatenate([X_train_con, X_train_cat], axis=-1)
    X_test  = np.concatenate([X_test_con, X_test_cat], axis=-1)

    del X_train_con, X_train_cat
    del X_test_con, X_test_cat

    joblib.dump(scaler, "scaler_dep.pkl")

In [ ]:
X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test  = y_test.astype('float32')

In [ ]:
#-- get shapes
print(f'X_train : {X_train.shape}')
print(f'X_test  : {X_test.shape}')
print(f'y_train : {y_train.shape}')
print(f'y_test  : {y_test.shape}')

X_train : (1433368, 90, 20)
X_test  : (327794, 90, 20)
y_train : (1433368,)
y_test  : (327794,)


In [ ]:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))

(array([0., 1.], dtype=float32), array([1320702,  112666]))
(array([0., 1.], dtype=float32), array([311659,  16135]))


In [ ]:
seed_value = 42

np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

# **3. TRAINING MODEL**

In [ ]:
#-- initialize hyperparameters
hyperparameters = {
    "model"                 : "gru",
    "epoch"                 : 50,
    "batch"                 : 128,
    "neurons"               : 8,
    "step_in"               : step_in,
    "step_out"              : step_out,
    "optimizer"             : "Adam",
    "constant_l"            : 0.0,
    "class_weight"          : False,
    "loss_function"         : "BCE",
    "regularization"        : "None",
    "learning_rate"         : 1e-5,
    "dropout_rate"          : 0.0,
    "dropout_rate_layer"    : 0.0,
    "dropout_rate_recurrent": 0.0,
}

In [ ]:
#-- others hyperparameters
es_params = {
    "monitor"              : "val_recall",
    "patience"             : 100,
    "restore_best_weights" : True
}

## **3.1 CREATE MODEL**

In [ ]:
def ExtractContext(x):
    return x[:, -1:, :]

def ExtractContextShape(input_shape):
    return (input_shape[0], 1, input_shape[2])

def GetContext(x):
    return tf.squeeze(x, axis=1)

def GetContextShape(input_shape):
    return (input_shape[0], input_shape[2])

In [ ]:
if hyperparameters["model"] == 'rnn':
    inputs = Input(shape=(timesteps, n_features))
    rnn    = SimpleRNN(hyperparameters["neurons"], return_sequences=False)(inputs)
    output = Dense(1, activation='sigmoid')(rnn)
    model  = Model(inputs=inputs, outputs=output)

In [ ]:
if hyperparameters["model"] == 'gru':
    inputs = Input(shape=(timesteps, n_features))
    gru    = GRU(hyperparameters["neurons"], return_sequences=False)(inputs)
    output = Dense(1, activation='sigmoid')(gru)
    model  = Model(inputs=inputs, outputs=output)

I0000 00:00:1749549510.273684      89 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [ ]:
if hyperparameters["model"] == 'lstm':
    inputs = Input(shape=(timesteps, n_features))
    lstm   = LSTM(hyperparameters["neurons"], return_sequences=False)(inputs)
    output = Dense(1, activation='sigmoid')(lstm)
    model  = Model(inputs=inputs, outputs=output)

In [ ]:
if hyperparameters["model"] == 'bilstm':
    inputs = Input(shape=(timesteps, n_features))
    lstm   = Bidirectional(LSTM(hyperparameters["neurons"], return_sequences=False))(inputs)
    output = Dense(1, activation='sigmoid')(lstm)
    model  = Model(inputs=inputs, outputs=output)

In [ ]:
if hyperparameters["model"] == 'lstm-at':
    inputs    = Input(shape=(timesteps, n_features))
    lstm      = LSTM(hyperparameters["neurons"], return_sequences=True)(inputs)
    query     = Lambda(ExtractContext, output_shape=ExtractContextShape)(lstm)
    attention = Attention()([query, lstm])
    context   = Lambda(GetContext, output_shape=GetContextShape)(attention)
    output    = Dense(1, activation='sigmoid')(context)
    model     = Model(inputs=inputs, outputs=output)

In [ ]:
if hyperparameters["model"] == 'rnn-at':
    inputs    = Input(shape=(timesteps, n_features))
    rnn       = SimpleRNN(hyperparameters["neurons"], return_sequences=True)(inputs)
    query     = Lambda(ExtractContext, output_shape=ExtractContextShape)(rnn)
    attention = Attention()([query, rnn])
    context   = Lambda(GetContext, output_shape=GetContextShape)(attention)
    output    = Dense(1, activation='sigmoid')(context)
    model     = Model(inputs=inputs, outputs=output)

In [ ]:
if hyperparameters["model"] == 'gru-at':
    inputs    = Input(shape=(timesteps, n_features))
    gru       = GRU(hyperparameters["neurons"], return_sequences=True)(inputs)
    query     = Lambda(ExtractContext, output_shape=ExtractContextShape)(gru)
    attention = Attention()([query, gru])
    context   = Lambda(GetContext, output_shape=GetContextShape)(attention)
    output    = Dense(1, activation='sigmoid')(context)
    model     = Model(inputs=inputs, outputs=output)

In [ ]:
if hyperparameters["model"] == 'bilstm-at':
    inputs    = Input(shape=(timesteps, n_features))
    lstm      = Bidirectional(LSTM(hyperparameters["neurons"], return_sequences=True))(inputs)
    query     = Lambda(ExtractContext, output_shape=ExtractContextShape)(lstm)
    attention = Attention()([query, lstm])
    context   = Lambda(GetContext, output_shape=GetContextShape)(attention)
    output    = Dense(1, activation='sigmoid')(context)
    model     = Model(inputs=inputs, outputs=output)

## **3.2 COMPILE MODEL**

In [ ]:
#-- add callbacks
early_stopping = EarlyStopping(**es_params)

In [ ]:
#-- model params
model_params = {
    "loss"      : BinaryCrossentropy(),
    "optimizer" : Adam(learning_rate=hyperparameters["learning_rate"], clipnorm=1.0),
    "metrics"   : [
        "accuracy",
        AUC(name = "auc"),
        Precision(name = "precision"),
        Recall(name = "recall")
    ]
}

In [ ]:
#-- add class weight
if (hyperparameters["class_weight"]):
    class_weights = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train.flatten())
    class_weights = {0: 2, 1: 1}

    print(class_weights)

In [ ]:
#-- compile model
model.compile(**model_params)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 90, 20)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 8)                   │             720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 729 (2.85 KB)

 Trainable params: 729 (2.85 KB)

 Non-trainable params: 0 (0.00 B)

## **3.3 DATA MODELLING**

In [ ]:
#-- training params
model_params = {
    "x": X_train,
    "y": y_train,
    "epochs": hyperparameters["epoch"],
    "callbacks": [early_stopping],
    "batch_size": hyperparameters["batch"],
    "validation_data": (X_test, y_test)
}

if hyperparameters["class_weight"]:
    model_params["class_weight"] = class_weights

In [ ]:
#-- train models
history = model.fit(**model_params)

## **3.4 MODEL EVALUATION**

In [ ]:
#-- save models
model_name = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_{hyperparameters['model']}_{hyperparameters['step_in']}_{hyperparameters['step_out']}"
model_path = os.path.join(os.getcwd(), f'{model_name}.keras')
model.save(model_path)

In [ ]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs >= 0.5).astype(int)

In [ ]:
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred):.3f}")
print(f"AP: {average_precision_score(y_test, y_pred):.3f}")

In [ ]:
# evaluate model
f1          = f1_score(y_test, y_pred, average='macro')
recall      = recall_score(y_test, y_pred, zero_division=1, average='macro')
accuracy    = accuracy_score(y_test, y_pred)
precision   = precision_score(y_test, y_pred, zero_division=1, average='macro')
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy Score   : {accuracy}')
print(f'Precision Score  : {precision}')
print(f'Recall Score     : {recall}')
print(f'F1 Score         : {f1}')
print(f'Confusion matrix : \n {conf_matrix}')

In [ ]:
# display confusion matrix
fig, ax = plt.subplots(figsize=(7, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=[0,1])
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.title("Confusion Matrix")
plt.savefig(f'{well_name}_conf_matrix.png')
plt.show()

In [ ]:
# Extract accuracy from the history object
training_recall = history.history['recall']
validation_recall = history.history['val_recall']
training_precision = history.history['precision']
validation_precision = history.history['val_precision']

# Plot training & validation accuracy
fig = plt.figure(figsize=(6.7,6))
plt.plot(training_recall)
plt.plot(validation_recall)
plt.plot(training_precision, linestyle='--', alpha=0.3)
plt.plot(validation_precision, linestyle='--', alpha=0.3)
plt.title('Training and Validation')
plt.ylabel('Value', size=10)
plt.xlabel('Epoch', size=10)
plt.legend(['Train', 'Validation'], loc='lower right')
plt.savefig(f'{well_name}_training_loss.png')
plt.show()

In [ ]:
# visualize models prediction vs real data
fig = plt.figure(figsize=(6.55,6))

plt.plot(y_test, label='Real data')
plt.plot(y_pred, label=hyperparameters["model"], alpha=0.6)
plt.title('Comparison between predicted data and test data')
plt.xlabel('Values', fontsize = 10)
plt.ylabel('Class', fontsize = 10)
plt.legend(loc='upper right', fontsize = 10)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.savefig(f'{well_name}_prediction.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6.7,6))
plt.title("Prediction Probablities [test data]")
plt.hist(y_pred_probs)
plt.savefig(f'{well_name}_prediction_prob.png')
plt.show()

In [ ]:
#-- find best treshold
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)

f1_scores      = 2 * (precision * recall) / ((precision + recall) + 1e-5)
best_threshold = thresholds[f1_scores.argmax()]
best_threshold

In [ ]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs >= best_threshold).astype(int)

In [ ]:
# evaluate model
f1          = f1_score(y_test, y_pred, average='macro')
recall      = recall_score(y_test, y_pred, zero_division=1, average='macro')
accuracy    = accuracy_score(y_test, y_pred)
precision   = precision_score(y_test, y_pred, zero_division=1, average='macro')
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy Score   : {accuracy}')
print(f'Precision Score  : {precision}')
print(f'Recall Score     : {recall}')
print(f'F1 Score         : {f1}')
print(f'Confusion matrix : \n {conf_matrix}')

In [ ]:
# display confusion matrix
fig, ax = plt.subplots(figsize=(7, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=[0,1])
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# visualize models prediction vs real data
fig = plt.figure(figsize=(6.55,6))

plt.plot(y_test, label='Real data')
plt.plot(y_pred, label=hyperparameters["model"], alpha=0.6)
plt.title('Comparison between predicted data and test data')
plt.xlabel('Values', fontsize = 10)
plt.ylabel('Class', fontsize = 10)
plt.legend(loc='upper right', fontsize = 10)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.show()